## Scrap the latest data presented in the website

the purpose of this notbook to scrap the latest data in the website, and thats because the website was in maintenance and update 

## Scrap text data

In [16]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag

def extract_clean_text(element):
    """Recursively extract clean visible text, preserving structure."""
    texts = []

    for child in element.descendants:
        if isinstance(child, NavigableString):
            text = child.strip()
            if text:
                texts.append(text)
        elif child.name in ["br", "p", "div", "section", "article", "h1", "h2", "h3", "h4", "li"]:
            texts.append("\n")

    joined = " ".join(texts)
    lines = [line.strip() for line in joined.split("\n") if line.strip()]
    return "\n\n".join(lines)


def scrape_full_structured(url):
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")

    for tag in soup(["script", "style", "noscript", "header", "footer", "nav"]):
        tag.decompose()

    body = soup.body
    if not body:
        body = soup

    content = extract_clean_text(body)
    return content


def save_text(filename, text):
    with open(filename, "w", encoding="utf8") as f:
        f.write(text)



In [17]:
url = "https://ensate.uae.ac.ma/formation-continue-3/"
text = scrape_full_structured(url)
save_text("data_final/Formation-continue-2025-2026.txt", text)

print("Saved all structured text.")

Saved all structured text.


c:\Users\PC Paradise\Desktop\Work\Projects\ENSATe-chatbot\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ensate.uae.ac.ma'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Scrap Documents

In [18]:
import os
import requests
from bs4 import BeautifulSoup

def download_all_pdfs(url, output_folder="data_final/emploi-temps"):
    # Create folder if not exists
    os.makedirs(output_folder, exist_ok=True)

    # Fetch HTML
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find PDF links
    pdf_links = []

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.lower().endswith(".pdf"):
            # Handle relative URLs
            if href.startswith("http"):
                pdf_links.append(href)
            else:
                # Generate absolute URL if needed
                from urllib.parse import urljoin
                pdf_links.append(urljoin(url, href))

    print("Found:", len(pdf_links), "data_final/emploi-temps")

    # Download each PDF
    for link in pdf_links:
        filename = link.split("/")[-1]

        print("Downloading:", filename)

        pdf_data = requests.get(link, verify=False).content

        with open(os.path.join(output_folder, filename), "wb") as f:
            f.write(pdf_data)

    print("Done, all PDFs saved in:", output_folder)



In [19]:
url = "https://ensa-tetouan.ac.ma/emploi-du-temps" 
download_all_pdfs(url)

ConnectionError: HTTPSConnectionPool(host='ensa-tetouan.ac.ma', port=443): Max retries exceeded with url: /emploi-du-temps (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002BD031CC5D0>: Failed to resolve 'ensa-tetouan.ac.ma' ([Errno 11002] getaddrinfo failed)"))